In [163]:
import random
random.seed(42) # Define um fator aleatoriedade para todo o script. Fazendo com que essa aleatoriedade possa ser comparada.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder

train = pd.read_csv('train.csv', header=(0))
train_attributes = list(train.columns)
print("Número de linhas: ", len(train))
train.head(10)

Número de linhas:  1460


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,1,2008,WD,Normal,118000


In [164]:
test = pd.read_csv('test.csv', header=(0))
test_id = test["Id"]
test_attributes = list(test.columns)
test.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
5,1466,60,RL,75.0,10000,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal
6,1467,20,RL,NaN,7980,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,GdPrv,Shed,500,3,2010,WD,Normal
7,1468,60,RL,63.0,8402,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal
8,1469,20,RL,85.0,10176,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2010,WD,Normal
9,1470,20,RL,70.0,8400,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,4,2010,WD,Normal


## Limpeza e Preparação dos dados

In [165]:
# MSSubClass: Identifica o tipo de habitação objeto da venda.

# MSZoning: Identifica a classificação geral da zona da venda.

# LotFrontage: se refere à medida linear, em pés, da frente do lote que está conectada à rua. Em outras palavras, é a extensão da propriedade ao longo da rua.

# LotArea: É a área total do terreno, medida em unidades de área quadrada nos Estados Unidos.

# Street: Essa variável descreve o tipo de via que proporciona acesso à propriedade, como por exemplo, rua pavimentada, estrada de terra, beco sem saída, entre outros.

# Alley: se refere ao tipo de acesso à propriedade através de um beco ou viela. Essa variável descreve o tipo de acesso secundário ou via de serviço que conecta a propriedade a uma via principal ou à parte traseira de outras propriedades. Isso pode incluir diferentes condições, como acesso pavimentado, cascalho, terra batida, ou até mesmo a ausência de acesso por viela.

# LotShape: Descreve a forma geral da propriedade, principalmente o que define a regularidade da casa. Em graus de regularidade

# LandCointour descreve a planura da propriedade. As caracteristicas definem diferentes formas topográficas.

# Utilities: Descreve o tipo de utilidades disponíveis na propriedade, como eletricidade, gás, água e esgoto.

# LotConfig:  Descreve a configuração do lote. Aqui estão as categorias e suas descrições. Essas categorias indicam diferentes configurações de localização do lote em relação às vias públicas e aos lotes adjacentes.

# LandSlope: Declive do terreno da propriedade - Suave, Moderado, Acentuado

# Neighborhood: Localizações físicas dentro dos limites da cidade de Ames - Bloomington Heights, Bluestem, Briardale, etc...

# Condition1: Proximidade a várias condições - Adjacente a rua arterial, Adjacente a rua de alimentação, Normal, Dentro de 200' da ferrovia Norte-Sul, Adjacente a ferrovia Norte-Sul, Próximo a recurso positivo fora do local - parque, área verde, etc...

# Condition2: Proximidade a várias condições (caso mais de uma esteja presente) - Igual Condition1

# BldgType: Tipo de habitação - Unifamiliar (Casa independente), Conversão bifamiliar; originalmente construída como uma única habitação, Duplex, Unidade de ponta de condomínio, Unidade interna de condomínio

# HouseStyle: Estilo de habitação - Uma história, Uma e meia história: 2º nível acabado, Basicamente isso é referindo sobre a casa.

# OverallQual: Avalia o material geral e acabamento da casa - 10: Muito Excelente, 9: Excelente, etc...

# OverallCond: Avalia a condição geral da casa - 10: Muito Excelente, 9: Excelente, 8: Muito Bom, etc..

# YearBuilt: Data original de construção

# YearRemodAdd: Data de remodelação (a mesma da data de construção se não houve remodelação ou adições)

# RoofStyle: Tipo de telhado - Plano, Gable (Duas águas), Gambrel (Celeiro), Hip (Quatro águas), Mansard (Mansarda), Shed (Galpão)

# RoofMatl: Material do telhado - Argila ou Telha, Telha Padrão (Composta), Membrana, Metal, Rolo, Cascalho e Alcatrão, Telhas de Madeira, Aglomerados de Madeira

# Exterior1st: Revestimento exterior da casa - Telhas de Amianto, Telhas de Asfalto, Tijolo Comum, Tijolo Aparente, Bloco de Concreto, Placa de Cimento, Painel de Madeira, etc...

# Exterior2nd: Revestimento exterior da casa (caso haja mais de um material) - Telhas de Amianto, Telhas de Asfalto, Tijolo Comum, Tijolo Aparente, Bloco de Concreto, Placa de Cimento, Painel de Madeira, etc...

# MasVnrType: Tipo de revestimento de alvenaria - Tijolo Comum, Tijolo Aparente, Bloco de Concreto, Nenhum, Pedra

# ExterQual: Avalia a qualidade do material na parte exterior - Excelente, Bom, Médio/Típico, Regular, Ruim

# ExterCond: Avalia a condição atual do material na parte exterior - Excelente, Bom, Médio/Típico, Regular, Ruim

# Foundation: Tipo de fundação - Tijolo e Telha, Bloco de Concreto, Concreto Lançado, Laje, Pedra, Madeira

# BsmtQual: Avalia a altura do porão - Excelente (100+ polegadas), Bom (90-99 polegadas), etc...

# BsmtCond: Avalia a condição geral do porão - Excelente, Bom, Típico - leve umidade permitida, etc...

# BsmtExposure: Refere-se às paredes do nível térreo ou do jardim - Boa exposição, Exposição Média (níveis divididos ou hall de entrada geralmente pontuam médio ou acima), Exposição Mínima, Sem Exposição, Sem porão

# BsmtFinType1: Classificação da área acabada do porão - Ótimos Quartos de Vida, Quartos de Vida Médios, Quartos de Vida Abaixo da Média, Sala de Recreação Média, Qualidade Baixa, Inacabado, Sem porão

# BsmtFinSF1: Pés quadrados acabados do tipo 1

# BsmtFinType2: Classificação da área acabada do porão (se houver vários tipos) - Ótimos Quartos de Vida, Quartos de Vida Médios, Quartos de Vida Abaixo da Média, Sala de Recreação Média, Qualidade Baixa, Inacabado, Sem porão

# BsmtFinSF2: Pés quadrados acabados do tipo 2

# BsmtUnfSF: Pés quadrados não acabados da área do porão

# TotalBsmtSF: Pés quadrados totais da área do porão

# Heating: Tipo de aquecimento - Forno de chão, Forno a gás forçado com ar quente, Aquecimento a gás com água quente ou vapor, Forno de gravidade, Aquecimento a água quente ou vapor que não seja a gás, Forno de parede

# HeatingQC: Qualidade e condição do aquecimento - Excelente, Bom, Médio/Típico, Regular, Ruim

# CentralAir: Ar condicionado central - Não, Sim

# Electrical: Sistema elétrico - Padrão com disjuntores e fiação Romex, Caixa de fusíveis acima de 60 AMP e toda fiação Romex (Médio), Caixa de fusíveis de 60 AMP e principalmente fiação Romex (Regular), Caixa de fusíveis de 60 AMP e principalmente fiação de knob & tube (Ruim), Misto

# 1stFlrSF: Pés quadrados do primeiro andar

# 2ndFlrSF: Pés quadrados do segundo andar

# LowQualFinSF: Pés quadrados acabados de baixa qualidade (todos os pisos)

# GrLivArea: Área de estar acima do solo (terreno) em pés quadrados

# BsmtFullBath: Banheiros completos no porão

# BsmtHalfBath: Banheiros de meio no porão

# FullBath: Banheiros completos acima do solo

# HalfBath: Meio banhos acima do solo

# Bedroom: Quartos acima do solo (não inclui quartos do porão)

# Kitchen: Cozinhas acima do solo

# KitchenQual: Qualidade da cozinha - Excelente, Bom, Médio/Típico, Regular, Ruim

# TotRmsAbvGrd: Total de quartos acima do solo (não inclui banheiros)

# Functional: Funcionalidade da casa (Assume típica a menos que deduções sejam justificadas) - Funcionalidade Típica, Deduções Menores 1, Deduções Menores 2, Deduções Moderadas, Deduções Principais 1, Deduções Principais 2, Danos Severos, Apenas Salvamento

# Fireplaces: Número de lareiras

# FireplaceQu: Qualidade da lareira - Excelente - Lareira de Alvenaria Excepcional, Bom - Lareira de Alvenaria no nível principal, Médio - Lareira pré-fabricada na área de estar principal ou Lareira de Alvenaria no porão, Regular - Lareira pré-fabricada no porão, Ruim - Fogão Ben Franklin, Sem lareira

# GarageType: Localização da garagem - Mais de um tipo de garagem, Anexado à casa, Garagem no porão, Embutido (parte da casa - geralmente tem um quarto acima da garagem), Garagem coberta, Separado da casa, Sem garagem

# GarageYrBlt: Ano em que a garagem foi construída

# GarageFinish: Acabamento interior da garagem - Acabado, Acabamento Rugoso, Inacabado, Sem garagem

# GarageCars: Tamanho da garagem em capacidade de carros

# GarageArea: Tamanho da garagem em pés quadrados

# GarageQual: Qualidade da garagem - Excelente, Bom, Médio/Típico, Regular, Ruim, Sem garagem

# GarageCond: Condição da garagem - Excelente, Bom, Médio/Típico, Regular, Ruim, Sem garagem

# PavedDrive: Entrada pavimentada - Pavimentada, Pavimento parcial, Terra/Cascalho

# WoodDeckSF: Área do deque de madeira em pés quadrados

# OpenPorchSF: Área do alpendre aberto em pés quadrados

# EnclosedPorch: Área do alpendre fechado em pés quadrados

# 3SsnPorch: Área do alpendre de três estações em pés quadrados

# ScreenPorch: Área da varanda da tela em pés quadrados

# PoolArea: Área da piscina em pés quadrados

# PoolQC: Qualidade da piscina - Excelente, Bom, Médio/Típico, Regular, Sem piscina

# Fence: Qualidade da cerca - Bom, Mínimo, Bom madeira, Madeira/Metal mínimo, Sem cerca

# MasVnrArea: Área de revestimento de alvenaria em pés quadrados

# MiscFeature: Característica diversa não coberta em outras categorias - Elevador, 2ª Garagem (se não descrita na seção de garagem), Outro, Galpão (mais de 100 pés quadrados), Quadra de tênis, Nenhum

# MiscVal: Valor $ da característica diversa

# MoSold: Mês de venda (MM)

# YrSold: Ano de venda (AAAA)

# SaleType: Tipo de venda - Escritura de garantia - Convencional, Escritura de garantia - Dinheiro, Escritura de garantia - Empréstimo VA, Nova construção, Escritura de Oficial de Justiça/Herança, Contrato - 15% de entrada termos regulares, Contrato - Baixa entrada e baixo interesse, Contrato - Baixo interesse, Contrato - Baixa entrada, Outro

# SaleCondition: Condição de venda - Venda normal, Venda anormal - troca, execução hipotecária, venda curta, Compra de terreno adjacente, Alocação - duas propriedades vinculadas com escrituras separadas, tipicamente condomínio com unidade de garagem, Venda entre membros da família, Parcial - Casa não foi concluída quando foi avaliada pela última vez (associada a novas casas)

In [166]:
nulls = train.isnull().sum().sort_values(ascending=False)
nulls.head(20)

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
MasVnrType       872
FireplaceQu      690
LotFrontage      259
GarageYrBlt       81
GarageCond        81
GarageType        81
GarageFinish      81
GarageQual        81
BsmtFinType2      38
BsmtExposure      38
BsmtQual          37
BsmtCond          37
BsmtFinType1      37
MasVnrArea         8
Electrical         1
Id                 0
dtype: int64

Como há muitas informações faltantes, eu vou excluir colunas que possuem mais de 50% de faltantes. E o restante será preenchida com a moda, média ou qualquer outra coisa.

Claro que não são informações que eu considero relevantes

LotFrontage = Medida do tamanho da frente. Será substituida pela média

GarageYrBlt = Ano de construção da garagem. Será substituida pela moda

GarageCond = Condição Garagem. Será substituida pela Moda

GarageType  = Tipo Garagem. Será substituida pela Moda

GarageFinish = Acabamento da Garagem. Será substituida pela Moda

GarageQual = Qualidade Garagem. Será substituida pela Moda

BsmtExposure = Será substituida pela Moda

BsmtFinType2 = Será substituida pela Moda

BsmtCond = Será substituida pela Moda

BsmtQual = Será substituida pela Média

BsmtFinType1 = Será substituida pela Moda

MasVnrArea  = Será substituida pela Média

Electrical = Será substituida pela Moda

In [167]:
def funcao_limpeza_de_dados(dataframe: pd.DataFrame):

    dataframe = dataframe.drop(columns=["Id", "PoolQC", "MiscFeature", "Alley", "Fence", "MasVnrType", "FireplaceQu"])
    
    dataframe['GarageYrBlt'] = dataframe['GarageYrBlt'].fillna(dataframe['GarageYrBlt'].mode()[0])
    dataframe['GarageCond'] = dataframe['GarageCond'].fillna(dataframe['GarageCond'].mode()[0])
    dataframe['GarageType'] = dataframe['GarageType'].fillna(dataframe['GarageType'].mode()[0])
    dataframe['GarageFinish'] = dataframe['GarageFinish'].fillna(dataframe['GarageFinish'].mode()[0])
    dataframe['GarageQual'] = dataframe['GarageQual'].fillna(dataframe['GarageQual'].mode()[0])
    dataframe['BsmtExposure'] = dataframe['BsmtExposure'].fillna(dataframe['BsmtExposure'].mode()[0])
    dataframe['BsmtFinType2'] = dataframe['BsmtFinType2'].fillna(dataframe['BsmtFinType2'].mode()[0])
    dataframe['BsmtCond'] = dataframe['BsmtCond'].fillna(dataframe['BsmtCond'].mode()[0])
    dataframe['BsmtQual'] = dataframe['BsmtQual'].fillna(dataframe['BsmtQual'].mode()[0])
    dataframe['BsmtFinType1'] = dataframe['BsmtFinType1'].fillna(dataframe['BsmtFinType1'].mode()[0])
    dataframe['MasVnrArea'] = dataframe['MasVnrArea'].fillna(dataframe['MasVnrArea'].mean())
    dataframe['Electrical'] = dataframe['Electrical'].fillna(dataframe['Electrical'].mode()[0])
    dataframe['LotFrontage'] = dataframe['LotFrontage'].fillna(dataframe['LotFrontage'].mean())

    return dataframe

train = funcao_limpeza_de_dados(train)
train.head(10)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.000000,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.000000,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,68.000000,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,60.000000,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,84.000000,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000
5,50,RL,85.000000,14115,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,320,0,0,700,10,2009,WD,Normal,143000
6,20,RL,75.000000,10084,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,8,2007,WD,Normal,307000
7,60,RL,70.049958,10382,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,228,0,0,0,350,11,2009,WD,Normal,200000
8,50,RM,51.000000,6120,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,205,0,0,0,0,4,2008,WD,Abnorml,129900
9,190,RL,50.000000,7420,Pave,Reg,Lvl,AllPub,Corner,Gtl,...,0,0,0,0,0,1,2008,WD,Normal,118000


In [168]:
#Conferindo números nulos
nulls = train.isnull().sum().sort_values(ascending=False)
nulls.head(20)

MSSubClass      0
GarageYrBlt     0
Fireplaces      0
Functional      0
TotRmsAbvGrd    0
KitchenQual     0
KitchenAbvGr    0
BedroomAbvGr    0
HalfBath        0
FullBath        0
BsmtHalfBath    0
BsmtFullBath    0
GrLivArea       0
LowQualFinSF    0
2ndFlrSF        0
1stFlrSF        0
Electrical      0
GarageType      0
GarageFinish    0
MSZoning        0
dtype: int64

In [169]:
def separando_em_numericos_e_categoricos(dataframe):

    columns_type = pd.DataFrame({'Column': dataframe.columns, 'Type': dataframe.dtypes}).reset_index(drop=True)

    objects_columns = columns_type[columns_type["Type"] == "object"]["Column"].values
    numerics_columns = columns_type[columns_type["Type"] != "object"]["Column"].values

    return objects_columns, numerics_columns

objects_columns, numerics_columns = separando_em_numericos_e_categoricos(train)

display(train[objects_columns].head())
display(train[numerics_columns].head())

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,Electrical,KitchenQual,Functional,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Normal
1,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,...,SBrkr,TA,Typ,Attchd,RFn,TA,TA,Y,WD,Normal
2,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Normal
3,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,...,SBrkr,Gd,Typ,Detchd,Unf,TA,TA,Y,WD,Abnorml
4,RL,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,...,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Normal


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,61,0,0,0,0,0,2,2008,208500
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,298,0,0,0,0,0,0,5,2007,181500
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,42,0,0,0,0,0,9,2008,223500
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,35,272,0,0,0,0,2,2006,140000
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,192,84,0,0,0,0,0,12,2008,250000


In [170]:
OHE = OneHotEncoder(categories="auto", handle_unknown="ignore", sparse_output=False)
categoric_train = OHE.fit_transform(train[objects_columns])
categoric_train = pd.DataFrame(categoric_train,columns=OHE.get_feature_names_out(objects_columns))
categoric_train.head()

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [171]:
train = categoric_train.join(train[numerics_columns])
train.head()

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0,61,0,0,0,0,0,2,2008,208500
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,298,0,0,0,0,0,0,5,2007,181500
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0,42,0,0,0,0,0,9,2008,223500
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0,35,272,0,0,0,0,2,2006,140000
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,192,84,0,0,0,0,0,12,2008,250000


In [172]:
sale_price_column = train.pop("SalePrice") 
train["SalePrice"] = sale_price_column #Jogando a Coluna para Último

In [173]:
data_train = train.to_numpy()
nrow,ncol = data_train.shape
y = data_train[:,-1]
X = data_train[:,0:ncol-1]

In [174]:
from sklearn.model_selection import train_test_split

p = 0.2
x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size = p, random_state = 42)

In [175]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_regression


grid = { 
    'n_estimators': [500],
    'max_features': ['sqrt'],
    'max_depth' : [7],
    'random_state' : [42]
}

best_model = GridSearchCV(estimator=RandomForestRegressor(), param_grid=grid, cv= 5)
best_model.fit(x_train, y_train)


GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [7], 'max_features': ['sqrt'],
                         'n_estimators': [500], 'random_state': [42]})

In [176]:
import numpy as np
from sklearn import metrics

y_pred = best_model.predict(x_valid)

print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_valid, y_pred))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_valid, y_pred))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_valid, y_pred)))
mape = np.mean(np.abs((y_valid - y_pred) / np.abs(y_valid)))
print('Mean Absolute Percentage Error (MAPE):', round(mape * 100, 2))
print('Accuracy:', round(100*(1 - mape), 2))



Mean Absolute Error (MAE): 21118.974935525937
Mean Squared Error (MSE): 1280506490.5507057
Root Mean Squared Error (RMSE): 35784.16536054328
Mean Absolute Percentage Error (MAPE): 13.61
Accuracy: 86.39


### Aplicar o modelo achado ao conjunto de teste e assim obter o resultado para submissão no Kaggle

In [177]:
test.isnull().sum().sort_values(ascending=False)

PoolQC           1456
MiscFeature      1408
Alley            1352
Fence            1169
MasVnrType        894
                 ... 
Electrical          0
1stFlrSF            0
2ndFlrSF            0
LowQualFinSF        0
SaleCondition       0
Length: 80, dtype: int64

In [178]:
test = funcao_limpeza_de_dados(test)
test

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,120,0,0,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,36,0,0,0,0,12500,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,34,0,0,0,0,0,3,2010,WD,Normal
3,60,RL,78.0,9978,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,36,0,0,0,0,0,6,2010,WD,Normal
4,120,RL,43.0,5005,Pave,IR1,HLS,AllPub,Inside,Gtl,...,82,0,0,144,0,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,RM,21.0,1936,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,6,2006,WD,Normal
1455,160,RM,21.0,1894,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,24,0,0,0,0,0,4,2006,WD,Abnorml
1456,20,RL,160.0,20000,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,9,2006,WD,Abnorml
1457,85,RL,62.0,10441,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,32,0,0,0,0,700,7,2006,WD,Normal


In [179]:
objects_columns, numerics_columns = separando_em_numericos_e_categoricos(test)

display(test[objects_columns].head())
display(test[numerics_columns].head())

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,Electrical,KitchenQual,Functional,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,RH,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,...,SBrkr,TA,Typ,Attchd,Unf,TA,TA,Y,WD,Normal
1,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,...,SBrkr,Gd,Typ,Attchd,Unf,TA,TA,Y,WD,Normal
2,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,...,SBrkr,TA,Typ,Attchd,Fin,TA,TA,Y,WD,Normal
3,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,...,SBrkr,Gd,Typ,Attchd,Fin,TA,TA,Y,WD,Normal
4,RL,Pave,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,...,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Normal


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,20,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,...,730.0,140,0,0,0,120,0,0,6,2010
1,20,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,...,312.0,393,36,0,0,0,0,12500,6,2010
2,60,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,...,482.0,212,34,0,0,0,0,0,3,2010
3,60,78.0,9978,6,6,1998,1998,20.0,602.0,0.0,...,470.0,360,36,0,0,0,0,0,6,2010
4,120,43.0,5005,8,5,1992,1992,0.0,263.0,0.0,...,506.0,0,82,0,0,144,0,0,1,2010


In [180]:
categoric_test: pd.DataFrame = OHE.transform(test[objects_columns])
categoric_test = pd.DataFrame(categoric_test,columns=OHE.get_feature_names_out(objects_columns))
categoric_test.head()
categoric_test

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1455,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1456,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1457,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [181]:
test = categoric_test.join(test[numerics_columns])
test.head()

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,730.0,140,0,0,0,120,0,0,6,2010
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,312.0,393,36,0,0,0,0,12500,6,2010
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,482.0,212,34,0,0,0,0,0,3,2010
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,470.0,360,36,0,0,0,0,0,6,2010
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,506.0,0,82,0,0,144,0,0,1,2010


In [182]:
data_test = test.to_numpy()
nrow,ncol = data_test.shape
X_test = data_test
X_test

array([[0.000e+00, 0.000e+00, 1.000e+00, ..., 0.000e+00, 6.000e+00,
        2.010e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.250e+04, 6.000e+00,
        2.010e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 3.000e+00,
        2.010e+03],
       ...,
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 9.000e+00,
        2.006e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 7.000e+02, 7.000e+00,
        2.006e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 1.100e+01,
        2.006e+03]])

In [183]:
y_pred_test = best_model.predict(X_test)
y_pred_test
prediction = pd.DataFrame()
prediction["Id"] = test_id
prediction["SalePrice"] = y_pred_test
prediction

,Id,SalePrice
0,1461,130565.596824
1,1462,149337.509579
2,1463,185098.673537
3,1464,191773.247962
4,1465,204339.143936
...,...,...
1454,2915,106093.344151
1455,2916,110004.743380
1456,2917,158587.758407
1457,2918,137737.639657


In [185]:
prediction.to_csv("prediction_HouseSales.csv", index=False)